In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

c:\Users\SSTLabs\miniconda3\envs\wahoman\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "beomi/OPEN-SOLAR-KO-10.7B"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards: 100%|██████████| 8/8 [00:18<00:00,  2.28s/it]


In [31]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "은행금리에 대해 설명해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(
    model_inputs,
    max_new_tokens=100,  # 응답 길이 제한
    num_return_sequences=1,  # 하나의 응답만 생성
    temperature=0.1,  # 응답의 다양성 조절
    do_sample=True
)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

은행금리에 대해 설명해줘 [/INST]
은행금리는 은행이 예금자에게 예금에 대한 대가로 지급하는 이자율이다. 은행은 예금자에게 이자를 지급하는 대신 예금자로부터 예금에 대한 대가로 이자를 지급받는다. 은행은 예금자에게 지급한 이자를 예금자에게 지급받은 이자로 충당한다. 은행이 예금자에게 지급하는 이자를 예금금리라고 하고, 은행이 예금자로부터 지급받는 이자를 대출금리라고 한다. 은행은 예금금리와 대출금리의 차이를 통해 수익을 창출한다


In [32]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [33]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    return_full_text=True,
    max_new_tokens=300
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [35]:
loader = PyPDFLoader("D:/여형구/test.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [36]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


In [37]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
result = rag_chain.invoke("yolo 몇 모델을 사용했어?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: - iv -<그림24>라벨링데이터예시 29
<그림25>VisualStudioCode환경에서YOLOv5모델학습30
<그림26>YOLOv5x모델학습결과(그림) 32
<그림27>학습모델의F1_curve(좌상),P_curve(우상),PR_curve(좌
하),R_curve(우하) 33
<그림28>True예시(좌),False중클래스만잘못예측한예시(우)34
<그림29>테스트데이터셋디텍팅결과(그림) 35
<그림30>실시간Detecting구현예시 36 / 출처: D:/여형구/test.pdf - 6 


주어진 근거: - 21 -3)YOLOv5[23]
YOLO는YouOnlyLookOnce의약자로객체탐지분야에서가장많이알
려진모델이다.객체탐지모델에는두가지접근방식이존재하는데예측결
과를도출하기까지몇번의단계를거치는지에따라One-Stage방식과
Two-Stage방식으로나누어진다.YOLO는이름그대로One-Stage방식으로
객체를탐지하는모델로객체의경계상자(boundingbox)와클래스레이블
(classlabel)을직접예측하는모델이다.방식의특성상구현이간단하고속도
가빠르다는장점이있어YOLO모델이실시간객체탐지가가능하게한다.
그림16One-StagevsTwo-Stage방식비교16)
YOLO모델은이미지가입력되면우선가로세로를동일한그리드영역으
로나눈다.그후그리드영역에대해서어디에객체가존재하는지바운딩박
스와해당박스에대한신뢰도점수를예측한다.이와동시에객체의종류를
예측하는Classification작업이진행된다.최종적으로정확도가높은박스들만
남기고NMS(Non-Maximum Suppression)[22]알고리즘을이용해선별하여 / 출처: D:/여형구/test.pdf - 29 


주어진 근거: - 30 -3)모델학습환경준비
본연구에서는적은양의데이터를가지고최선의결과를도출하기위해
YOLOv5의4가지모델중가장크고무거운YOLOv5x모델을사용하였다.
또한,학습파라미터를표6과같이설정하여학습을진행하였다.
종류 값
LearningRate(학습률) 0.01
BatchSize(배치크기) 8
Epochs(에폭) 50
Opt

In [54]:
print(f"\n답변: {result['text']}")


답변: 
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

[Document(page_content='- iv -<그림24>라벨링데이터예시 29\n<그림25>VisualStudioCode환경에서YOLOv5모델학습30\n<그림26>YOLOv5x모델학습결과(그림) 32\n<그림27>학습모델의F1_curve(좌상),P_curve(우상),PR_curve(좌\n하),R_curve(우하) 33\n<그림28>True예시(좌),False중클래스만잘못예측한예시(우)34\n<그림29>테스트데이터셋디텍팅결과(그림) 35\n<그림30>실시간Detecting구현예시 36', metadata={'source': 'D:/여형구/test.pdf', 'page': 6}), Document(page_content='- 21 -3)YOLOv5[23]\nYOLO는YouOnlyLookOnce의약자로객체탐지분야에서가장많이알\n려진모델이다.객체탐지모델에는두가지접근방식이존재하는데예측결\n과를도출하기까지몇번의단계를거치는지에따라One-Stage방식과\nTwo-Stage방식으로나누어진다.YOLO는이름그대로One-Stage방식으로\n객체를탐지하는모델로객체의경계상자(boundingbox)와클래스레이블\n(classlabel)을직접예측하는모델이다.방식의특성상구현이간단하고속도\n가빠르다는장점이있어YOLO모델이실시간객체탐지가가능하게한다.\n그림16One-StagevsTwo-Stage방식비교16)\nYOLO모델은이미지가입력되면우선가로세로를동일한그리드영역으\n로나눈다.그후그리드영역에대해서어디에객체가존재하는지바운딩박\n스와해당박스에대한신뢰도점수를예측한다.이와동시에객체의종류를\n예측하는Classification작업이진행된다.최종적으로정확도가높은박스들만\n남기고NMS(Non-Maximum Suppression)[22]알고리즘을이용해선별하여', metadata={'source': 'D:/여형구/test.pd